# PIL - Gross domestic product (GDP) at current market prices by NUTS 2 region
https://ec.europa.eu/eurostat/web/main/data/database



Contiene il PIL (GDP) e le sue principali componenti (consumi, investimenti, esportazioni, ecc.) per tutte le regioni europee di livello NUTS2, su base annuale.

- https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/nama_10r_2gdp
- https://ec.europa.eu/eurostat/databrowser/product/page/NAMA_10R_2GDP

Il file è strutturato secondo lo standard SDMX-JSON. Contiene:

1. structure
Metadati sulle dimensioni (es. geo, na_item, unit, time_period) e valori (etichette, codici, posizione degli indici).

2. dataSets
Array con i valori effettivi. I dati sono compattati in forma:

json
Copia
Modifica
"value": {
  "0:0:0:0": 124500.0,
  "0:0:0:1": 128000.0,
  ...
}
Ogni chiave "x:y:z:w" è un riferimento alla combinazione di indici nelle dimensioni (es. unit:geo:na_item:time), e il valore è il dato (es. PIL in milioni di euro).

3. dimensions / attributes
Contiene la lista dei valori per ciascuna dimensione e i loro ID testuali.

## analisi
nama_10r_2gdp è il dataset GDP a prezzi correnti per regioni NUTS 2 (il secondo livello delle regioni europee), di Eurostat


| Dataset         | Livello NUTS | Dimensioni            | Contenuto                           |
| --------------- | ------------ | --------------------- | ----------------------------------- |
| `nama_10r_2gdp` | NUTS2        | freq, unit, geo, time | Solo **PIL totale**                 |
| `nama_10r_3gdp` | NUTS3        | freq, unit, geo, time | Solo **PIL totale** a livello NUTS3 |


per trovare altre dimensioni:
- 🗂️ nama_10r_3gdp → solo PIL
- 🗂️ nama_10r_3gvar → value added by industry
- 🗂️ oppure: nama_10r_3popgdp → PIL pro capite già calcolato

In [1]:
import requests, pandas as pd

- nama_10r_2gdp - nama_10r_3gdp - nama_10r_2gvagr - namq_10_gdp - gdp nama_10r_3gdp - naidq_10_gdp - sts_inppd_m - sts_copr_m
- sts_inppd_m - sts_trtu_m - tipsna10 - tipslm40 - tipslm20 - nama_10_lp_

In [2]:
dataset = 'namq_10_gdp'   
url = f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{dataset}?format=JSON&na_item=B1GQ&unit=CLV10_MEUR&s_adj=SCA"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("✅ Connessione riuscita.")
    print("📦 Chiavi:", data.keys())
    print("📐 Dimensioni disponibili:", data["dimension"].keys())
    print("🔢 Numero di valori:", len(data["value"]))
else:
    print("❌ Errore:", response.status_code)
    print(response.text)


✅ Connessione riuscita.
📦 Chiavi: dict_keys(['version', 'class', 'label', 'source', 'updated', 'value', 'status', 'id', 'size', 'dimension', 'extension'])
📐 Dimensioni disponibili: dict_keys(['freq', 'unit', 's_adj', 'na_item', 'geo', 'time'])
🔢 Numero di valori: 5106


|dataset|dataset|
|-|-|
|nama_10r_2|| 
|nama_10r_3gdp|| 
|nama_10r_2gvagr||
|sts_inppd_m||

|sts_inppd_|Prezzi alla Produzione dei Prodotti Industriali|


In [25]:
# dimensioni disponibili
list(data["dimension"].keys())

['freq', 'unit', 'geo', 'time']

In [26]:
# Guarda quante categorie ci sono per ogni dimensione
for dim, details in data["dimension"].items():
    print(f"{dim}: {len(details['category']['index'])} items")

freq: 1 items
unit: 1 items
geo: 1790 items
time: 24 items


In [23]:
labels = data["dimension"]["na_item"]["category"]["label"]
for code, label in sorted(labels.items()):
    print(f"{code}: {label}")

B1GQ: Gross domestic product at market prices


### 📏 **Significato delle unità (`unit`)**

| Codice              | Descrizione                                                          | Tipo                        |
| ------------------- | -------------------------------------------------------------------- | --------------------------- |
| `MIO_EUR`           | Milioni di euro correnti                                             | Valore assoluto             |
| `EUR_HAB`           | Euro per abitante (PIL pro capite)                                   | Valore pro capite           |
| `EUR_HAB_EU27_2020` | Euro per abitante in % della media EU27 (base 2020 = 100)            | Indicatore relativo         |
| `PPS_EU27_2020_HAB` | PIL pro capite in standard di potere d’acquisto (PPS), base EU27=100 | Valore comparabile          |
| `PPS_HAB_EU27_2020` | PPS per abitante in % della media EU27                               | Indicatore relativo         |
| `MIO_PPS_EU27_2020` | Milioni di PPS (standard di potere d’acquisto), EU27 base 2020       | Valore assoluto comparabile |
| `MIO_NAC`           | Milioni di valuta nazionale corrente (utile per paesi non euro)      | Valore nazionale            |


### 🧠 Quale usare?

| Obiettivo                                | Unità consigliata                         |
| ---------------------------------------- | ----------------------------------------- |
| Confronto interno (tra regioni italiane) | `MIO_EUR` o `EUR_HAB`                     |
| Confronto internazionale (con UE)        | `PPS_EU27_2020_HAB` o `EUR_HAB_EU27_2020` |
| Misura reale del PIL per dimensione      | `MIO_EUR`                                 |
| Pro capite reale e comparabile           | `PPS_EU27_2020_HAB`                       |


Nello script passare il parametro `unit=EUR_HAB` per filtrare direttamente il tipo di unità desiderato.



se invece usiamo **nama_10r_2gvagr**

#### 📐 `na_item` (tipo di variabile economica)

| Codice | Significato                     |
| ------ | ------------------------------- |
| `B1GQ` | **PIL** a prezzi di mercato     |
| `B1G`  | **Valore Aggiunto Lordo (GVA)** |

#### 📏 `unit` (unità di misura)

| Codice    | Significato                                           |
| --------- | ----------------------------------------------------- |
| `MIO_EUR` | Milioni di euro (valore assoluto)                     |
| `PCH_PRE` | Variazione percentuale rispetto al periodo precedente |
| `I15`     | Indice (base 2015=100)                                |



### 📌 Come usare questa combinazione

#### 🔹 1. **PIL della Puglia in milioni di euro**

```http
.../nama_10r_2gvagr?geo=ITF4&na_item=B1GQ&unit=MIO_EUR&format=JSON
```

#### 🔹 2. **GVA della Puglia in milioni di euro**

```http
.../nama_10r_2gvagr?geo=ITF4&na_item=B1G&unit=MIO_EUR&format=JSON
```

#### 🔹 3. **Indice GVA base 2015 della Puglia**

```http
.../nama_10r_2gvagr?geo=ITF4&na_item=B1G&unit=I15&format=JSON
```

#### 🔹 4. **Variazione % del PIL della Puglia**

```http
.../nama_10r_2gvagr?geo=ITF4&na_item=B1GQ&unit=PCH_PRE&format=JSON
```

In [22]:
geo_labels = data["dimension"]["geo"]["category"]["label"]

# Stampa ordinata per codice
for code, label in sorted(geo_labels.items()):
    print(f"{code}: {label}")

AL: Albania
AT: Austria
BE: Belgium
BG: Bulgaria
CH: Switzerland
CY: Cyprus
CZ: Czechia
DE: Germany
DK: Denmark
EA: Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)
EA20: Euro area – 20 countries (from 2023)
EE: Estonia
EL: Greece
ES: Spain
EU27_2020: European Union - 27 countries (from 2020)
FI: Finland
FR: France
HR: Croatia
HU: Hungary
IE: Ireland
IS: Iceland
IT: Italy
LT: Lithuania
LU: Luxembourg
LV: Latvia
ME: Montenegro
MK: North Macedonia
MT: Malta
NL: Netherlands
NO: Norway
PL: Poland
PT: Portugal
RO: Romania
RS: Serbia
SE: Sweden
SI: Slovenia
SK: Slovakia
TR: Türkiye
UK: United Kingdom


In [ ]:
# Parsing delle dimensioni
dimensions = data["dimension"]
freq_labels = dimensions["freq"]["category"]["label"]
unit_labels = dimensions["unit"]["category"]["label"]
geo_labels = dimensions["geo"]["category"]["label"]
time_labels = dimensions["time"]["category"]["label"]

# Recupera l’ordine delle dimensioni
dimension_order = data["id"]  # es. ['freq', 'unit', 'geo', 'time']
dimension_sizes = data["size"]

# Ricostruisci tutte le combinazioni di indici
from itertools import product

freq_keys = list(freq_labels.keys())
unit_keys = list(unit_labels.keys())
geo_keys = list(geo_labels.keys())
time_keys = list(time_labels.keys())

combinations = list(product(freq_keys, unit_keys, geo_keys, time_keys))

# Ricostruisci i valori
records = []
for i, (f, u, g, t) in enumerate(combinations):
    if str(i) in data["value"]:
        records.append({
            "freq": freq_labels[f],
            "unit": unit_labels[u],
            "geo": geo_labels[g],
            "time": time_labels[t],
            "value": data["value"][str(i)]
        })
df = pd.DataFrame(records)
(df.sample(10))

,freq,unit,geo,time,value
62193,Annual,Euro per inhabitant,Vukovarsko-srijemska županija,2020,8500.00
134189,Annual,Million units of national currency,Lasithi,2007,1383.01
102834,Annual,Euro per inhabitant in percentage of the EU27 ...,Isernia,2005,91.00
89207,Annual,Euro per inhabitant in percentage of the EU27 ...,"Delmenhorst, Kreisfreie Stadt",2019,76.00
241786,Annual,"Purchasing power standard (PPS, EU27 from 2020...",Ruse,2010,31.00
77212,Annual,Euro per inhabitant,Sremska oblast,2016,5000.00
97893,Annual,Euro per inhabitant in percentage of the EU27 ...,Tenerife,2022,62.00
263124,Annual,"Purchasing power standard (PPS, EU27 from 2020...",Napoli,2015,65.00
244539,Annual,"Purchasing power standard (PPS, EU27 from 2020...",Ravensburg,2021,136.00
258418,Annual,"Purchasing power standard (PPS, EU27 from 2020...",Indre,2000,87.00


In [28]:
df.to_csv('D:/dashboard/Eurostat_PilProCapite.csv', index = False, decimal =',', sep ='|')

In [ ]:
dataset = 'nama_10r_2gdp'   
url = f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{dataset}?format=JSON"

response = requests.get(url)
data = response.json()

# Parsing delle dimensioni
dimensions = data["dimension"]
freq_labels = dimensions["freq"]["category"]["label"]
unit_labels = dimensions["unit"]["category"]["label"]
geo_labels = dimensions["geo"]["category"]["label"]
time_labels = dimensions["time"]["category"]["label"]

# Recupera l’ordine delle dimensioni
dimension_order = data["id"]  # es. ['freq', 'unit', 'geo', 'time']
dimension_sizes = data["size"]

# Ricostruisci tutte le combinazioni di indici
from itertools import product

freq_keys = list(freq_labels.keys())
unit_keys = list(unit_labels.keys())
geo_keys = list(geo_labels.keys())
time_keys = list(time_labels.keys())

combinations = list(product(freq_keys, unit_keys, geo_keys, time_keys))

# Ricostruisci i valori
records = []
for i, (f, u, g, t) in enumerate(combinations):
    if str(i) in data["value"]:
        records.append({
            "freq": freq_labels[f],
            "unit": unit_labels[u],
            "geo": geo_labels[g],
            "time": time_labels[t],
            "value": data["value"][str(i)]
        })
df = pd.DataFrame(records)
df.to_csv('{dir}Eurostat_{dataset}.csv', index = False, decimal =',', sep ='|')

## decodifica colonna na_items

In [5]:
import pandas as pd
import os

# Cartella contenente i CSV
folder_path = 'data'

# Lista dei file CSV
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Dizionario per raccogliere i valori unici di 'na_item' per ciascun file
na_item_uniques = {}

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path, sep='|')
        if 'na_item' in df.columns:
            unique_values = df['na_item'].dropna().unique().tolist()
            na_item_uniques[file] = unique_values
    except Exception as e:
        print(f"Errore nel file {file}: {e}")

# Stampa dei risultati
for file, items in na_item_uniques.items():
    print(f"\n📄 {file} — unique('na_item'):")
    for item in items:
        print(f"  - {item}")



📄 Eurostat_nama_10r_2gvagr.csv — unique('na_item'):
  - Gross domestic product at market prices
  - Value added, gross

📄 Eurostat_namq_10_gdp.csv — unique('na_item'):
  - Gross domestic product at market prices
  - Value added, gross
  - Final consumption expenditure
  - Final consumption expenditure of general government
  - Individual consumption expenditure of general government
  - Collective consumption expenditure of general government
  - Household and NPISH final consumption expenditure
  - Final consumption expenditure of households
  - Final consumption expenditure of NPISH
  - Actual individual consumption
  - Gross capital formation
  - Gross fixed capital formation
  - Exports of goods and services
  - Exports of goods
  - Exports of services
  - Imports of goods and services
  - Imports of goods
  - Imports of services
  - Taxes less subsidies on products
  - Taxes on products
  - Subsidies on products
  - Final consumption expenditure and gross capital formation
  - Fi

## ricerca dataset con province italiane

In [12]:
import os
import pandas as pd

# Cartella contenente i file CSV
folder_path = 'data'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

province_files = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        # Legge solo la colonna 'geo'
        geo_col = pd.read_csv(file_path, sep = '|', usecols=['geo'])['geo'].dropna().astype(str).unique()
        # Cerca codici NUTS3 (province italiane) — es. ITC44, ITH55
        nuts3_like = [g for g in geo_col if g.startswith('IT') and len(g) == 5]
        if nuts3_like:
            province_files.append({
                'file': file,
                'codici_provinciali': nuts3_like[:5]  # primi 5 per verifica
            })
    except Exception as e:
        print(f"⚠️ Errore nel file {file}: {e}")

# Stampa dei risultati
if province_files:
    print("\n✅ File CSV che contengono codici NUTS3 (province italiane):\n")
    for entry in province_files:
        print(f"📄 {entry['file']}")
        for codice in entry['codici_provinciali']:
            print(f"   - {codice}")
else:
    print("❌ Nessun file con codici provinciali (NUTS3) trovato.")



✅ File CSV che contengono codici NUTS3 (province italiane):

📄 Eurostat_nama_10r_3gdp.csv
   - ITC11
   - ITC12
   - ITC13
   - ITC14
   - ITC15


## estrazione dati provinciali 

In [ ]:
import pandas as pd

df = pd.read_csv("data/Eurostat_nama_10r_3gdp.csv", sep='|', decimal=',')
# a = df[df['geo'].str.match(r'^IT[A-Z0-9]{3}$')].copy() # Seleziona solo i record con codici geo di tipo NUTS3 (es. ITC44, ITF31)
a = df[df['geo'].str.startswith('IT')].copy()
province_codes = a['geo'].unique()
print(f"✅ Province italiane trovate: {len(province_codes)} codici NUTS3")
print(sorted(province_codes.tolist()))


✅ Province italiane trovate: 137 codici NUTS3
['IT', 'ITC', 'ITC1', 'ITC11', 'ITC12', 'ITC13', 'ITC14', 'ITC15', 'ITC16', 'ITC17', 'ITC18', 'ITC2', 'ITC20', 'ITC3', 'ITC31', 'ITC32', 'ITC33', 'ITC34', 'ITC4', 'ITC41', 'ITC42', 'ITC43', 'ITC44', 'ITC46', 'ITC47', 'ITC48', 'ITC49', 'ITC4A', 'ITC4B', 'ITC4C', 'ITC4D', 'ITF', 'ITF1', 'ITF11', 'ITF12', 'ITF13', 'ITF14', 'ITF2', 'ITF21', 'ITF22', 'ITF3', 'ITF31', 'ITF32', 'ITF33', 'ITF34', 'ITF35', 'ITF4', 'ITF43', 'ITF44', 'ITF45', 'ITF46', 'ITF47', 'ITF48', 'ITF5', 'ITF51', 'ITF52', 'ITF6', 'ITF61', 'ITF62', 'ITF63', 'ITF64', 'ITF65', 'ITG', 'ITG1', 'ITG11', 'ITG12', 'ITG13', 'ITG14', 'ITG15', 'ITG16', 'ITG17', 'ITG18', 'ITG19', 'ITG2', 'ITG2D', 'ITG2E', 'ITG2F', 'ITG2G', 'ITG2H', 'ITH', 'ITH1', 'ITH10', 'ITH2', 'ITH20', 'ITH3', 'ITH31', 'ITH32', 'ITH33', 'ITH34', 'ITH35', 'ITH36', 'ITH37', 'ITH4', 'ITH41', 'ITH42', 'ITH43', 'ITH44', 'ITH5', 'ITH51', 'ITH52', 'ITH53', 'ITH54', 'ITH55', 'ITH56', 'ITH57', 'ITH58', 'ITH59', 'ITI', 'ITI1', 'IT

In [15]:
a.sample(10)

,freq_code,freq_label,unit_code,unit_label,geo,geo_label,time_code,time_label,value
144463,A,Annual,MIO_NAC,Million units of national currency,ITI32,Ancona,2014,2014,13564.67
142611,A,Annual,MIO_NAC,Million units of national currency,ITF34,Avellino,2018,2018,7943.07
183129,A,Annual,MIO_PPS_EU27_2020,"Million purchasing power standards (PPS, EU27 ...",ITF12,Teramo,2003,2003,5876.15
65439,A,Annual,EUR_HAB,Euro per inhabitant,ITI22,Terni,2002,2002,21300.00
25319,A,Annual,MIO_EUR,Million euro,ITZZ,Extra-Regio NUTS 2,2014,2014,1493.96
225099,A,Annual,PPS_EU27_2020_HAB,"Purchasing power standard (PPS, EU27 from 2020...",ITH52,Parma,2007,2007,33900.00
143320,A,Annual,MIO_NAC,Million units of national currency,ITG2E,Nuoro,2008,2008,3812.36
25018,A,Annual,MIO_EUR,Million euro,ITI3,Marche,2015,2015,39890.41
22526,A,Annual,MIO_EUR,Million euro,ITC32,Savona,2004,2004,7110.45
143287,A,Annual,MIO_NAC,Million units of national currency,ITG2,Sardegna,2022,2022,38826.61


In [17]:
b = a['unit_label'].unique()
print(f"✅ Province italiane trovate: {len(b)} codici NUTS3")
print(sorted(b.tolist()))

✅ Province italiane trovate: 7 codici NUTS3
['Euro per inhabitant', 'Euro per inhabitant in percentage of the EU27 (from 2020) average', 'Million euro', 'Million purchasing power standards (PPS, EU27 from 2020)', 'Million units of national currency', 'Purchasing power standard (PPS, EU27 from 2020), per inhabitant', 'Purchasing power standard (PPS, EU27 from 2020), per inhabitant in percentage of the EU27 (from 2020) average']


# Backup

## estrazione massiva

Per ogni codice dataset nella lista:

Interroga l’API RESTful Eurostat

Estrae i dati (decodificati)

Salva un file CSV ./data/{codice}.csv

In [11]:
import requests
import pandas as pd
from itertools import product
import os

# Directory di salvataggio
output_dir = "./data"
os.makedirs(output_dir, exist_ok=True)

# Codici dei dataset Eurostat da scaricare
dataset_codes = [
    "nama_10r_2gdp", "nama_10r_3gdp", "nama_10r_2gvagr", "nama_10_lp_ulc",
    "naidq_10_gdp", "naida_10_gdp", "naidq_10_a10",
    "tipsna10", "tipslm40", "tipslm20"
]

for dataset in dataset_codes:
    print(f"\n⬇️ Scaricando dataset: {dataset}")
    url = f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{dataset}?format=JSON"

    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"❌ Errore HTTP {response.status_code} per {dataset}")
            continue

        data = response.json()
        dimensions = data["dimension"]
        dimension_order = data["id"]
        dimension_sizes = data["size"]

        # Etichette: mappa codice → etichetta leggibile
        labels = {
            dim: dimensions[dim]["category"]["label"]
            for dim in dimension_order
        }

        # Chiavi disponibili per ogni dimensione
        keys = {
            dim: list(labels[dim].keys())
            for dim in dimension_order
        }

        # Combinazioni di tutte le chiavi
        combinations = list(product(*(keys[dim] for dim in dimension_order)))

        # Ricostruzione dei record con codice + etichetta per ogni dimensione
        records = []
        for i, combo in enumerate(combinations):
            if str(i) in data["value"]:
                record = {}
                for dim, key in zip(dimension_order, combo):
                    record[f"{dim}_code"] = key
                    record[f"{dim}_label"] = labels[dim][key]
                record["value"] = data["value"][str(i)]
                records.append(record)

        # Costruisci DataFrame
        df = pd.DataFrame(records)

        if not df.empty:
            # Aggiungi alias 'geo' per compatibilità con altri script
            if 'geo_code' in df.columns:
                df.rename(columns={'geo_code': 'geo'}, inplace=True)

            # Salvataggio CSV
            output_path = os.path.join(output_dir, f"Eurostat_{dataset}.csv")
            df.to_csv(output_path, index=False, sep='|', decimal=',')
            print(f"✅ Salvato: {output_path}")
        else:
            print(f"⚠️ Nessun dato trovato in {dataset}")

    except Exception as e:
        print(f"❌ Errore per {dataset}: {e}")



⬇️ Scaricando dataset: nama_10r_2gdp
✅ Salvato: ./data\Eurostat_nama_10r_2gdp.csv

⬇️ Scaricando dataset: nama_10r_3gdp
✅ Salvato: ./data\Eurostat_nama_10r_3gdp.csv

⬇️ Scaricando dataset: nama_10r_2gvagr
✅ Salvato: ./data\Eurostat_nama_10r_2gvagr.csv

⬇️ Scaricando dataset: nama_10_lp_ulc
✅ Salvato: ./data\Eurostat_nama_10_lp_ulc.csv

⬇️ Scaricando dataset: naidq_10_gdp
✅ Salvato: ./data\Eurostat_naidq_10_gdp.csv

⬇️ Scaricando dataset: naida_10_gdp
✅ Salvato: ./data\Eurostat_naida_10_gdp.csv

⬇️ Scaricando dataset: naidq_10_a10
✅ Salvato: ./data\Eurostat_naidq_10_a10.csv

⬇️ Scaricando dataset: tipsna10
✅ Salvato: ./data\Eurostat_tipsna10.csv

⬇️ Scaricando dataset: tipslm40
✅ Salvato: ./data\Eurostat_tipslm40.csv

⬇️ Scaricando dataset: tipslm20
✅ Salvato: ./data\Eurostat_tipslm20.csv


In [ ]:
# solo la Puglia
url = ("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/nama_10r_2gdp?geo=ITF4&format=JSON")
response = requests.get(url)
print("Status:", response.status_code)

if response.status_code == 200:
    data = response.json()
    print("✅ Connessione riuscita.")
    print("📦 Totale valori:", len(data["value"]))
    print("🔳 Dimensioni:", data["dimension"].keys())
else:
    print("❌ Errore:", response.status_code)
    print(response.text)

Status: 200
✅ Connessione riuscita.
📦 Totale valori: 168
🔳 Dimensioni: dict_keys(['freq', 'unit', 'geo', 'time'])


In [ ]:
# Mappatura NUTS2 → NUTS1
nuts2_to_nuts1 = {
    "ITC1": "Nord-Ovest", "ITC2": "Nord-Ovest", "ITC3": "Nord-Ovest", "ITC4": "Nord-Ovest",
    "ITH1": "Nord-Est", "ITH2": "Nord-Est", "ITH3": "Nord-Est", "ITH4": "Nord-Est", "ITH5": "Nord-Est",
    "ITI1": "Centro", "ITI2": "Centro", "ITI3": "Centro", "ITI4": "Centro",
    "ITF1": "Sud", "ITF2": "Sud", "ITF3": "Sud", "ITF4": "Sud", "ITF5": "Sud", "ITF6": "Sud",
    "ITG1": "Isole", "ITG2": "Isole"
}

# Definizione del mapping Mezzogiorno / Resto d'Italia
nuts2_to_macroarea = {
    "ITF1": "Mezzogiorno", "ITF2": "Mezzogiorno", "ITF3": "Mezzogiorno",
    "ITF4": "Mezzogiorno", "ITF5": "Mezzogiorno", "ITF6": "Mezzogiorno",
    "ITG1": "Mezzogiorno", "ITG2": "Mezzogiorno",
    # Tutti gli altri → Resto d'Italia
    "ITC1": "Resto d'Italia", "ITC2": "Resto d'Italia", "ITC3": "Resto d'Italia", "ITC4": "Resto d'Italia",
    "ITH1": "Resto d'Italia", "ITH2": "Resto d'Italia", "ITH3": "Resto d'Italia", "ITH4": "Resto d'Italia", "ITH5": "Resto d'Italia",
    "ITI1": "Resto d'Italia", "ITI2": "Resto d'Italia", "ITI3": "Resto d'Italia", "ITI4": "Resto d'Italia"
}